## Multi-label classification  

Often, we may encouter data that can be classified into more than one categories (for example movie genre, items in an image).  
However, typical classification tasks involve predicting a single label, as they treat classes as being mutually exclusive.   

Multi-Label Classification is the supervised learning problem where an instance may be associated with multiple labels. This is opposed to the traditional task of single-label classification (i.e., multi-class, or binary) where each instance is only associated with a single class label. 

  

### Techniques   

There are two main categorizations of methods that can be used to solve for the multi-label classification problem  
* problem transformation methods and 
* algorithm adaptation methods 

In the first case the learning task is transformed into more or single-label classification tasks. 
In the second, the algorithms are adapted so that they can handle multi-label data.   


<br />

The dataset used here is the GoEmotions.  
This is a dataset released from Google and it containes the emotions detected in those texts.  
It is the largest manually annotated dataset of 58K English Reddit comments, labeled for 27 emotion categories or neutral.  
Find the paper on [arXiv.org](https://arxiv.org/abs/2005.00547)

In [1]:
import pathlib
import pandas as pd 

In [2]:
dataset = pathlib.Path.cwd() / 'Datasets/train.tsv'
df = pd.read_csv(dataset, sep='\t', header=None, names=['comment', 'label', 'id'])
df['label'] = df['label'].str.split(',')

In [3]:
emotion_list = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment',                     'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',                 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

enkman_mapping = {
        "anger": ["anger", "annoyance", "disapproval"],
        "disgust": ["disgust"],
        "fear": ["fear", "nervousness"],
        "joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire",                       "caring"],
        "sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
        "surprise": ["surprise", "realization", "confusion", "curiosity"],
        "neutral": ["neutral"],
        }
enkman_mapping_rev = {v:key for key, value in enkman_mapping.items() for v in value}

In [4]:
# function from Google Research analysis 
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [5]:
# add emotion label to the label ids
df['emotions'] = df['label'].apply(idx2class)

# use enkman mapping to reduce the emotions to a list of ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
df['mapped_emotions'] = df['emotions'].apply(lambda x: [enkman_mapping_rev[i] for i in x])

# fix issues where ['joy',' joy'] might appear
df.loc[df['mapped_emotions'].apply(len)>1, 'mapped_emotions'] = df.loc[df['mapped_emotions'].apply(len)>1, 'mapped_emotions'].apply(lambda x: [emotion for emotion in set(x)])

In [6]:
df[:14]

,comment,label,id,emotions,mapped_emotions
0,My favourite food is anything I didn't have to...,[27],eebbqej,[neutral],[neutral]
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i,[neutral],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj,[anger],[anger]
3,To make her feel threatened,[14],ed7ypvh,[fear],[fear]
4,Dirty Southern Wankers,[3],ed0bdzj,[annoyance],[anger]
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,[26],edvnz26,[surprise],[surprise]
6,Yes I heard abt the f bombs! That has to be wh...,[15],ee3b6wu,[gratitude],[joy]
7,We need more boards and to create a bit more s...,"[8, 20]",ef4qmod,"[desire, optimism]",[joy]
8,Damn youtube and outrage drama is super lucrat...,[0],ed8wbdn,[admiration],[joy]
9,It might be linked to the trust factor of your...,[27],eczgv1o,[neutral],[neutral]
